# Convolutional net with PyTorch for MNIST classification. 
## Inspired by  yunjey/pytorch-tutorial

In [40]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Configure device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)



cpu


In [41]:
# Load MNIST 
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

In [42]:
def build_dataloaders(train_dataset, test_dataset, batch_size):
    train_loader  = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)
    return train_loader, test_loader

## Model class

In [43]:

import torch.nn as nn

# Convolutional net with 2 layers
class ConvNet(nn.Module):
    def __init__(self, n_classes):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, n_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
    
    def train_model(self, train_loader, n_epochs):
        n_steps = len(train_loader)
        for epoch in range(n_epochs):
            for i, (images, labels) in enumerate(train_loader):
                images = images.to(device)
                labels = labels.to(device)
                
                # Forward pass
                outputs = self(images)
                loss = loss_func(outputs, labels)
                
                # Propagate backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                if (i+1) % 100 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                        .format(epoch+1, n_epochs, i+1, n_steps, loss.item()))
    
    def test_model(self, test_loader):
        self.eval() 
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = self(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

## Train and test

In [44]:
# Define hyper parameters
n_epochs = 5
n_classes = 10
batch_size = 100
learning_rate = 0.001

In [45]:
train_loader, test_loader = build_dataloaders(train_dataset, test_dataset, 
                                              batch_size)

In [46]:

# Define model
model = ConvNet(n_classes).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
model.train_model(train_loader, n_epochs)

Epoch [1/5], Step [100/600], Loss: 0.0956
Epoch [1/5], Step [200/600], Loss: 0.1065
Epoch [1/5], Step [300/600], Loss: 0.0443
Epoch [1/5], Step [400/600], Loss: 0.1244
Epoch [1/5], Step [500/600], Loss: 0.0394
Epoch [1/5], Step [600/600], Loss: 0.0903
Epoch [2/5], Step [100/600], Loss: 0.0396
Epoch [2/5], Step [200/600], Loss: 0.0107
Epoch [2/5], Step [300/600], Loss: 0.0147
Epoch [2/5], Step [400/600], Loss: 0.1593
Epoch [2/5], Step [500/600], Loss: 0.0145
Epoch [2/5], Step [600/600], Loss: 0.0419
Epoch [3/5], Step [100/600], Loss: 0.0159
Epoch [3/5], Step [200/600], Loss: 0.0182
Epoch [3/5], Step [300/600], Loss: 0.0423
Epoch [3/5], Step [400/600], Loss: 0.0661
Epoch [3/5], Step [500/600], Loss: 0.0780
Epoch [3/5], Step [600/600], Loss: 0.0879
Epoch [4/5], Step [100/600], Loss: 0.0144
Epoch [4/5], Step [200/600], Loss: 0.0623
Epoch [4/5], Step [300/600], Loss: 0.0182
Epoch [4/5], Step [400/600], Loss: 0.0332
Epoch [4/5], Step [500/600], Loss: 0.0109
Epoch [4/5], Step [600/600], Loss:

In [48]:

# Testing
model.test_model(test_loader)



Test Accuracy of the model on the 10000 test images: 99.06 %
